In [184]:
from pyspark import SparkContext, SparkConf
from operator import add
from collections import Counter 
import math 

In [185]:
def rdd_to_dictionary(rdd):
    return sc.parallelize(rdd).collectAsMap()

In [186]:
def write_json(dictionary,filename):
    import json  

    with open(filename, "w") as outfile:  
        json.dump(dictionary, outfile)    

In [187]:
def trimPunctuation(word):
    if word[0] in punctuation:
        return word[1:]
    elif word[-1] in punctuation:
        return word[:-1]
    else:
        return word

In [188]:
def remove_punct_stop_word(document):    
    document_data_bag = spark.read.text(document).rdd.map(lambda r: r[0])
    words_count = document_data_bag.flatMap(lambda x: x.strip().split())\
                .map(lambda x: x.lower()) \
                .map(lambda x: (x, 1))\
                .reduceByKey(add)\
                .filter(lambda x: x[1] > 1)\
                .filter(lambda x: len(x[0]) > 1)

    
#     words_without_punctuation = words_count.map(lambda x: (trimPunctuation(x[0]), x[1])).filter(lambda x: len(x[0]) > 1).reduceByKey(add)
    words_without_punctuation = words_count.map(lambda x: (x[0].strip('.,:;\'!?'), x[1])).reduceByKey(add)

#     print(words_without_punctuation.count())
    stop_words_data_bag = spark.read.text("data/stopwords.txt").rdd.map(lambda r: r[0])
    stop_words = stop_words_data_bag.flatMap(lambda x: x.split('\n'))\
                .map(lambda x: x.lower()).map(lambda x: (x, 1))\
                .reduceByKey(add)\
                .filter(lambda x: len(x[0]) > 0)
#     print(stop_words.count())
    words_without_punct_stop_words = words_without_punctuation.subtractByKey(stop_words)
#     print(words_without_punct_stop_words)
#     words_without_punct_stop_words_list = words_without_punct_stop_words.collect()
    return words_without_punct_stop_words


In [189]:
def generate_word_dictionary(doc_list):
    for i in doc_list:
        key = i[0]
        val = i[1]
        if key in total_words:
            total_words[key].append(val)
        else:
            total_words[key] = [val]


In [190]:
def find_word_occurences_in_all_docs(key):
    return len(total_words[key])

In [191]:
def calcualte_idf(document_count,key,val):
    print(find_word_occurences_in_all_docs(key))
    x = document_count // find_word_occurences_in_all_docs(key)
    idf = math.log(x)
    
    return idf

In [192]:
def calculate_td_idf(tf,idf):
    tf_idf = tf * idf
    return tf_idf

In [193]:
def get_top_40_tf_idf(doc):
#     print("printing")
#     print(doc.collect())
    words_tf_idf_rdd = doc.map(lambda x: (x[0],calculate_td_idf(x[1],calcualte_idf(N,x[0],x[1]))))
#     print(words_tf_idf_rdd.collect())
    top_5_idf = words_tf_idf_rdd.takeOrdered(5, key = lambda x: -x[1])
#     print(top_5_idf)
    for i in top_5_idf:
        top_40_td_idf_list.append(i)

In [194]:
conf = SparkConf().setMaster("local[*]")
sc = SparkContext.getOrCreate()

In [195]:
documents = ["data/4300-0.txt","data/pg36.txt", "data/pg514.txt","data/pg1497.txt","data/pg3207.txt","data/pg6130.txt","data/pg19033.txt","data/pg42671.txt"]
# documents = ["data/4300-0.txt","data/pg36.txt"]

total_words = {
    
}
punctuation = [".",",",":",";","'","!","?"]
doc_rdd = []
doc_list = []
top_40_td_idf_list = []
N = len(documents)
# print(N)


In [196]:
for doc in documents:   
    rdd = remove_punct_stop_word(doc)
    doc_rdd.append(rdd)
    doc_list.append(rdd.collect())
# print(doc_list)

In [197]:
for doc in doc_list:
    generate_word_dictionary(doc)
# print(total_words)


{'project': [87, 84, 84, 82, 82, 83, 82, 82], 'gutenberg': [24, 24, 24, 24, 24, 25, 24, 24], 'ebook': [9, 9, 9, 9, 9, 10, 9, 9], 'ulysses': [10, 118], 'james': [32, 3, 3], 'joyce': [4], 'use': [47, 16, 62, 153, 204, 31, 14, 23], 'anyone': [19, 10, 64, 4, 4, 4, 5, 4], 'anywhere': [11, 2, 11, 5, 2, 2, 2, 3], 'no': [624, 180, 442, 712, 964, 467, 31, 463], 'cost': [8, 3, 18, 2, 2, 9, 2, 6], 'almost': [31, 42, 32, 38, 29, 24, 3, 61], 'restrictions': [2, 2, 2, 2, 2, 2, 2, 2], 'whatsoever': [7, 2, 2, 2, 107, 2, 2, 2], 'you': [1909, 123, 1999, 1460, 193, 187, 143, 1232], 'copy': [21, 14, 21, 17, 13, 15, 12, 12], 'give': [162, 16, 133, 103, 158, 81, 7, 126], 'away': [176, 60, 311, 100, 91, 89, 14, 113], 're-use': [2, 2, 2, 2, 2, 2, 2, 2], 'under': [228, 74, 98, 155, 115, 40, 16, 37], 'terms': [33, 21, 24, 61, 28, 27, 21, 41], 'license': [15, 15, 16, 15, 15, 14, 15, 15], 'included': [8, 2, 2, 12, 5, 3, 2, 4], 'online': [4, 4, 4, 4, 4, 5, 6, 5], 'www.gutenberg.org': [2, 2, 2, 2, 4], 'title': [6, 

In [198]:
for doc in doc_rdd:
    get_top_40_tf_idf(doc)

[('stephen', 1000.211381548001), ('bloom', 614.1284019761115), ('j', 382.61724366908976), ('don’t', 359.74338671061156), ('dedalus', 332.71064666877373)]
[('martians', 326.4723220437342), ('martian', 160.11699870934734), ('pit', 109.51725452847136), ('woking', 103.97207708399179), ('heat-ray', 68.62157087543459)]
[('jo', 2474.5354345990045), ('meg', 1272.6182235080594), ('amy', 1135.3750817571904), ('laurie', 1087.547926298554), ('beth', 810.9822012551359)]
[('yes', 320.23399741869474), ('glaucon', 266.168517335019), ('republic', 224.57968650142226), ('plato', 223.88653932086234), ('thrasymachus', 174.6730895011062)]
[('soveraign', 1023.0852385064792), ('common-wealth', 867.1271228804915), ('onely', 856.7299151720923), ('civill', 673.7390595042668), ('kingdome', 592.6408393787532)]
[('thy', 644.6268779207492), ('hector', 499.0659700031606), ('jove', 467.18119969740314), ('skies', 326.4723220437342), ('trojan', 309.1436425297356)]
[('alice', 230.12486394590184), ('[illustration]', 45.74

In [199]:
# print(top_40_td_idf_list)

[('stephen', 1000.211381548001), ('bloom', 614.1284019761115), ('j', 382.61724366908976), ('don’t', 359.74338671061156), ('dedalus', 332.71064666877373), ('martians', 326.4723220437342), ('martian', 160.11699870934734), ('pit', 109.51725452847136), ('woking', 103.97207708399179), ('heat-ray', 68.62157087543459), ('jo', 2474.5354345990045), ('meg', 1272.6182235080594), ('amy', 1135.3750817571904), ('laurie', 1087.547926298554), ('beth', 810.9822012551359), ('yes', 320.23399741869474), ('glaucon', 266.168517335019), ('republic', 224.57968650142226), ('plato', 223.88653932086234), ('thrasymachus', 174.6730895011062), ('soveraign', 1023.0852385064792), ('common-wealth', 867.1271228804915), ('onely', 856.7299151720923), ('civill', 673.7390595042668), ('kingdome', 592.6408393787532), ('thy', 644.6268779207492), ('hector', 499.0659700031606), ('jove', 467.18119969740314), ('skies', 326.4723220437342), ('trojan', 309.1436425297356), ('alice', 230.12486394590184), ('[illustration]', 45.74771391

In [200]:
top_40_td_idf_dictionary = rdd_to_dictionary(top_40_td_idf_list)
print(top_40_td_idf_dictionary)

{'stephen': 1000.211381548001, 'bloom': 614.1284019761115, 'j': 382.61724366908976, 'don’t': 359.74338671061156, 'dedalus': 332.71064666877373, 'martians': 326.4723220437342, 'martian': 160.11699870934734, 'pit': 109.51725452847136, 'woking': 103.97207708399179, 'heat-ray': 68.62157087543459, 'jo': 2474.5354345990045, 'meg': 1272.6182235080594, 'amy': 1135.3750817571904, 'laurie': 1087.547926298554, 'beth': 810.9822012551359, 'yes': 320.23399741869474, 'glaucon': 266.168517335019, 'republic': 224.57968650142226, 'plato': 223.88653932086234, 'thrasymachus': 174.6730895011062, 'soveraign': 1023.0852385064792, 'common-wealth': 867.1271228804915, 'onely': 856.7299151720923, 'civill': 673.7390595042668, 'kingdome': 592.6408393787532, 'thy': 644.6268779207492, 'hector': 499.0659700031606, 'jove': 467.18119969740314, 'skies': 326.4723220437342, 'trojan': 309.1436425297356, 'alice': 230.12486394590184, '[illustration]': 45.747713916956386, 'rabbit': 41.58883083359672, 'caterpillar': 29.1121815

In [201]:
write_json(top_40_td_idf_dictionary,"sp4.json")

In [202]:
sc.stop()